In [1]:
# Dependencies
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
import re  #regular expression (from bs documentation)
import selenium
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float #Varchar?
Base = declarative_base()
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
# from config import remote_db_endpoint, remote_db_port
# from config import remote_hiv_dbname, remote_hiv_dbuser, remote_hiv_dbpwd
# from config import local_hiv_dbname, local_hiv_dbuser, local_hiv_dbpwd

Create engine to pass in Cloud MySQL Database Connection on AWS

In [ ]:
#Cloud MySQL Database Connection on AWS
#engine = create_engine(f'mysql://{remote_hiv_dbuser}:{remote_hiv_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_hiv_dbname}')

In [ ]:
#Create a remote database engine connection
#conn = engine.connect()

In [2]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

#Initiate browser w/ Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [157]:
drugs_all_hiv_drugs = ['emtricitabine-tenofovir/truvada-',
                       'efavirenz-emtricitabine-tenofovir/atripla-',
                       'ritonavir/norvir-',
                       'raltegravir/isentress-',
                       'darunavir/prezista-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir/stribild-',
                       'abacavir-dolutegravir-lamivudine/triumeq-',
                       'dolutegravir/tivicay-',
                       'emtricitabine-rilpivirine-tenofovir/complera-',
                       'abacavir-lamivudine/epzicom-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/genvoya-',
                       'atazanavir/reyataz-',
                       'emtricitabine-tenofovir-alafenamide/descovy-',
                       'lamivudine/',
                       'tenofovir/viread-',
                       'lamivudine/epivir-',
                       'lopinavir-ritonavir/kaletra-',
                       'efavirenz/sustiva-',
                       'efavirenz-emtricitabine-tenofovir/',
                       'etravirine/intelence-',
                       'ritonavir/',
                       'emtricitabine-tenofovir/',
                       'tenofovir/',
                       'abacavir/',
                       'bictegravir-emtricitabine-tenofovir-alafenamide/biktarvy-',
                       'abacavir-lamivudine/','efavirenz/',
                       'raltegravir/',
                       'darunavir/',
                       'dolutegravir/',
                       'abacavir/ziagen-',
                       'nevirapine/viramune-',
                       'atazanavir/',
                       'lamivudine-zidovudine/combivir-',
                       'cobicistat-darunavir/prezcobix-',
                       'emtricitabine/emtriva-',
                       'fosamprenavir/lexiva-',
                       'emtricitabine-rilpivirine-tenofovir-alafenamide/odefsey-',
                       'zidovudine/',
                       'nevirapine/',
                       'abacavir-lamivudine-zidovudine/trizivir-',
                       'indinavir/crixivan-',
                       'emtricitabine/',
                       'lopinavir-ritonavir/',
                       'stavudine/zerit-',
                       'abacavir-dolutegravir-lamivudine/',
                       'didanosine/',
                       'emtricitabine-rilpivirine-tenofovir/',
                       'etravirine/',
                       'indinavir/',
                       'lamivudine-zidovudine/',
                       'maraviroc/',
                       'stavudine/',
                       'didanosine/videx-',
                       'nelfinavir/viracept-',
                       'abacavir-lamivudine-zidovudine/',
                       'tipranavir/aptivus-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/',
                       'atazanavir-cobicistat/evotaz-',
                       'raltegravir/isentress-hd-',
                       'dolutegravir-rilpivirine/juluca-',
                       'bictegravir-emtricitabine-tenofovir-alafenamide/',
                       'cobicistat/',
                       'delavirdine/',
                       'doravirine/',
                       'efavirenz-lamivudine-tenofovir/',
                       'emtricitabine-rilpivirine-tenofovir-alafenamide/',
                       'emtricitabine-tenofovir-alafenamide/',
                       'enfuvirtide/',
                       'rilpivirine/',
                       'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/symtuza-',
                       'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/',
                       'dolutegravir-rilpivirine/']

In [158]:
brands = []
what_fors = []
reviews = []
dates_posted = []
star_ratings = []

for d in drugs_all_hiv_drugs:
    # URL of page to be scraped (so too many to type with, without brand and generic)
    url = f'https://www.drugs.com/comments/{d}for-hiv-infection.html'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    try:
        paging = soup.find('ul', class_='ddc-paging')
        paging_lis = paging.find_all('li')
        i = (len(paging_lis))
    except:
        i = 2


    for x in range(1,i):
        #test prints
        html = browser.html
        results = soup.body.find('div', class_='contentBox')
        branded = results.h2.text #why is this stripping the last letter?
        whats = "HIV"

        results2 = soup.body.find_all('div', class_='ddc-comment')
        for r in results2:
            #Get the review text
            review = r.p.text 
            
            #Get the date published
            try:
                date = r.find('span',class_='comment-date').text
            except:
                date = 'n/a'
            
            #Get the rating divide by 2 because it is a rating of 1-10 and we only want a rating 1-5
            try:
                rate = r.find('div', class_='rating-score').text #will have to divide by 2
                rating = int(float(rate))
            except:
                rating = np.nan  
            
            #Append scraped info to respective lists
            brands.append(branded)
            what_fors.append(whats)
            reviews.append(review)
            dates_posted.append(date)
            star_ratings.append(rating)
            
    print(f'{d}: Scrape Finished')
    time.sleep(10)
# print(brands,what_fors,reviews,dates_posted,star_ratings)

emtricitabine-tenofovir/truvada-: Scrape Finished
efavirenz-emtricitabine-tenofovir/atripla-: Scrape Finished
ritonavir/norvir-: Scrape Finished
raltegravir/isentress-: Scrape Finished
darunavir/prezista-: Scrape Finished
cobicistat-elvitegravir-emtricitabine-tenofovir/stribild-: Scrape Finished
abacavir-dolutegravir-lamivudine/triumeq-: Scrape Finished
dolutegravir/tivicay-: Scrape Finished
emtricitabine-rilpivirine-tenofovir/complera-: Scrape Finished
abacavir-lamivudine/epzicom-: Scrape Finished
cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/genvoya-: Scrape Finished
atazanavir/reyataz-: Scrape Finished
emtricitabine-tenofovir-alafenamide/descovy-: Scrape Finished
lamivudine/: Scrape Finished
tenofovir/viread-: Scrape Finished
lamivudine/epivir-: Scrape Finished
lopinavir-ritonavir/kaletra-: Scrape Finished
efavirenz/sustiva-: Scrape Finished
efavirenz-emtricitabine-tenofovir/: Scrape Finished
etravirine/intelence-: Scrape Finished
ritonavir/: Scrape Finished
emtricitab

In [159]:
#Scrape PrEP drugs
for d in drugs_prep_hiv_drugs:
    # URL of page to be scraped (so too many to type with, without brand and generic)
    url = f'https://www.drugs.com/comments/{d}for-pre-exposure-prophylaxis.html'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    try:
        paging = soup.find('ul', class_='ddc-paging')
        paging_lis = paging.find_all('li')
        i = (len(paging_lis))
    except:
        i = 2

    for x in range(1,i):
        #test prints
        html = browser.html
        results = soup.body.find('div', class_='contentBox')
        branded = results.h2.text #why is this stripping the last letter?
        whats = "PrEP"

        results2 = soup.body.find_all('div', class_='ddc-comment')
        for r in results2:
            #Get the review text
            review = r.p.text 
            
            #Get the date published
            try:
                date = r.find('span',class_='comment-date').text
            except:
                date = 'n/a'
            
            #Get the rating divide by 2 because it is a rating of 1-10 and we only want a rating 1-5
            try:
                rate = r.find('div', class_='rating-score').text #will have to divide by 2
                rating = (float(rate)/2)
            except:
                rating = np.nan  
            
            #Append scraped info to respective lists
            brands.append(branded)
            what_fors.append(whats)
            reviews.append(review)
            dates_posted.append(date)
            star_ratings.append(rating)
    
    print(f'{d}: Scrape Finished')
    time.sleep(10)
# print(brands,what_fors,reviews,dates_posted,star_ratings)

['Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Truvada Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Rating Summary', 'Atripla Ra

In [214]:
drugs_scrape_df = pd.DataFrame({'Drugs':brands,'Reason':what_fors,'Reviews':reviews,
                               'Date Posted':dates_posted,'Ratings':star_ratings,'Source':"drugs.com"})
drugs_scrape_df.head()

,Drugs,Reason,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...","April 11, 2018",1.0,drugs.com
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...","August 26, 2016",10,drugs.com
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...","October 18, 2015",3.0,drugs.com
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...","August 2, 2015",10,drugs.com
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...","February 2, 2015",10,drugs.com
5,Truvada Rating Summary,HIV,"""I started Truvada once a day along with Isent...","August 29, 2013",10,drugs.com
6,Truvada Rating Summary,HIV,"""I have been on the combination of Truvada/Pre...","December 24, 2011",9.0,drugs.com
7,Truvada Rating Summary,HIV,"""I started use Truvada, when Intelence affecte...","September 30, 2011",10,drugs.com
8,Truvada Rating Summary,HIV,"""My viral load has gone for 180,000 to undetec...","May 19, 2011",10,drugs.com
9,Truvada Rating Summary,HIV,"""After one year on Truvada in combination with...","April 19, 2010",9.0,drugs.com


In [26]:
# drugs_scrape_df.to_csv('drugs_scrape2.csv', index=False)
drugs_scrape_df = pd.read_csv('drugs_scrape.csv')
drugs_scrape_df.head()

,Drug,Reason,Reviews,Date Posted,Ratings,Source
0,Truvada,HIV,Doctor put me truvada as part of my medicatio...,"April 11, 2018",1.0,drugs.com
1,Truvada,HIV,No side effects experienced in almost two year...,"August 26, 2016",10.0,drugs.com
2,Truvada,HIV,I`ve been on truvada daily for more than 5 yea...,"October 18, 2015",3.0,drugs.com
3,Truvada,HIV,I have been taking Truvuda and Edurant combin...,"August 2, 2015",10.0,drugs.com
4,Truvada,HIV,Been on Truvada/Norvir/Prezista combination fo...,"February 2, 2015",10.0,drugs.com


In [27]:
#Clean up scraped data
drugs_scrape_df['Drug'] = drugs_scrape_df['Drug'].str.replace(r'Rating Summary','')
drugs_scrape_df['Drug'] = drugs_scrape_df['Drug'].str.replace(r'Reviews for','')
drugs_scrape_df['Drug'] = drugs_scrape_df['Drug'].str.strip()
drugs_scrape_df['Reviews'] = drugs_scrape_df['Reviews'].str.replace(r'"','')
drugs_scrape_df['Date Posted'] = drugs_scrape_df['Date Posted'].apply(lambda x: pd.to_datetime(x,))
drugs_scrape_df['Ratings'] = drugs_scrape_df['Ratings'].apply(lambda x: float(x))
# drugs_scrape_df['Ratings'] = drugs_scrape_df['Ratings'].apply(lambda x: x/2)
drugs_scrape_df


,Drug,Reason,Reviews,Date Posted,Ratings,Source
0,Truvada,HIV,Doctor put me truvada as part of my medicatio...,2018-04-11,1.0,drugs.com
1,Truvada,HIV,No side effects experienced in almost two year...,2016-08-26,10.0,drugs.com
2,Truvada,HIV,I`ve been on truvada daily for more than 5 yea...,2015-10-18,3.0,drugs.com
3,Truvada,HIV,I have been taking Truvuda and Edurant combin...,2015-08-02,10.0,drugs.com
4,Truvada,HIV,Been on Truvada/Norvir/Prezista combination fo...,2015-02-02,10.0,drugs.com
5,Truvada,HIV,I started Truvada once a day along with Isentr...,2013-08-29,10.0,drugs.com
6,Truvada,HIV,I have been on the combination of Truvada/Prez...,2011-12-24,9.0,drugs.com
7,Truvada,HIV,"I started use Truvada, when Intelence affected...",2011-09-30,10.0,drugs.com
8,Truvada,HIV,"My viral load has gone for 180,000 to undetect...",2011-05-19,10.0,drugs.com
9,Truvada,HIV,After one year on Truvada in combination with ...,2010-04-19,9.0,drugs.com


In [ ]:
#First found all unique values in table
#drugs_scrape_df.Drugs.unique()

#Dictionary with unique values to replace with brand drug names
drug_replacements_dict = {'Cobicistat/elvitegravir/emtricitabine/tenofovir alafenamide':'Genvoya',
                          'Abacavir/dolutegravir/lamivudine':'Triumeq',
                          'Efavirenz/emtricitabine/tenofovir':'Atripla',
                          'Bictegravir/emtricitabine/tenofovir alafenamide':'Biktarvy',
                          'Emtricitabine/rilpivirine/tenofovir':'Complera',
                          'Dolutegravir':'Tivicay','Raltegravir':'Isentress',
                          'Emtricitabine/tenofovir':'Truvada','Efavirenz':'Sustiva',
                          'Abacavir/lamivudine':'Epzicom','Emtricitabine/rilpivirine/tenofovir alafenamide':'Odefsey',
                          'Darunavir':'Prezista','Nevirapine':'Viramune','Emtricitabine/tenofovir alafenamide':'Descovy',
                          'Lopinavir/ritonavir':'Kaletra','Enfuvirtide':'Fuzeon',
                          'Dolutegravir/rilpivirine':'Juluca','Maraviroc':'Selzentry',
                          'Lamivudine/zidovudine':'Combivir','Etravirine':'Intelence',
                          'Lamivudine':'Epivir','Rilpivirine':'Edurant','Ritonavir':'Norvir',
                          'Abacavir/lamivudine/zidovudine':'Trizivir','Atazanavir':'Reyataz',
                          'Abacavir':'Ziagen','Cobicistat/darunavir/emtricitabine/tenofovir alafenamide':'Symtuza'
                         }
# 
drugs_scrape_df['Drug'] = drugs_scrape_df['Drug'].map(drug_replacements_dict).fillna(drugs_scrape_df['Drug'])
drugs_scrape_df

In [187]:
#Drug classes and abbrv's NRTs:Nucleoside Reverse Transcriptase Inhibitor; NNRTIs: Non-Nucleoside
#Reverse Transcriptase Inhibitor; PIs: Protease Inhibitor; CAs:CCR5 Antagonist; INSTIs: Integrase
# Inhibitor; MCCs: Multi-Class Combination; PrEPs: Pre-Exposure Prophylaxis; PEPs: Post-Exposure Prophylaxis
hiv_drug_classes = ['NRTs','NNRTIs','PIs','CAs','INSTIs','MCCs','PrEPs','PEPs']
nrts = ['Combivir', 'Descovy', 'Emtriva', 'Epivir', 'Epizicom','Hivid','Trizivir',
            'Videx','Viread','Zerit','Ziagen','Truvada']
nnrtis = ['Edurant','Intelence','Rescriptor','Pifeltro','Sustiva','Viramune']
pis = ['Agenerase','Aptivus','Crixivan','Invirase','Kaletra','Lexiva','Norvir',
      'Prezista','Reyataz','Viracept']
cas = ['Selzentry']
instis = ['Isentress','Tivicay','Vitekta']
mccs = ['Atripla','Biktarvy','Cimduo','Complera','Delstrigo','Dovato','Genvoya',
       'Evotaz','Juluca','Odefsey','Prezcobix','Symfi','Stribild','Symtuza','Triumeq']
preps = ['Truvada','Descovy']

In [224]:
#Classify each drug
drugs_scrape_df['Classify'] = pd.np.where(drugs_scrape_df.Drugs.isin(mccs), 'MMC',
                    pd.np.where(drugs_scrape_df.Drugs.isin(instis), 'INSTI',
                    pd.np.where(drugs_scrape_df.Drugs.isin(nnrtis), 'NNRTI',
                    pd.np.where(drugs_scrape_df.Drugs.isin(nrts), 'NRT',
                    pd.np.where(drugs_scrape_df.Drugs.isin(pis), 'PI', 'None'            
                    )))))

drugs_scrape_df.loc[drugs_scrape_df['Reason'].str.contains('PrEP'), 'Classify'] = 'PrEP'

drugs_scrape_df

,Drugs,Reason,Reviews,Date Posted,Ratings,Source,Classify
0,Truvada,HIV,Doctor put me truvada as part of my medicatio...,2018-04-11,1.0,drugs.com,NRT
1,Truvada,HIV,No side effects experienced in almost two year...,2016-08-26,10.0,drugs.com,NRT
2,Truvada,HIV,I`ve been on truvada daily for more than 5 yea...,2015-10-18,3.0,drugs.com,NRT
3,Truvada,HIV,I have been taking Truvuda and Edurant combin...,2015-08-02,10.0,drugs.com,NRT
4,Truvada,HIV,Been on Truvada/Norvir/Prezista combination fo...,2015-02-02,10.0,drugs.com,NRT
5,Truvada,HIV,I started Truvada once a day along with Isentr...,2013-08-29,10.0,drugs.com,NRT
6,Truvada,HIV,I have been on the combination of Truvada/Prez...,2011-12-24,9.0,drugs.com,NRT
7,Truvada,HIV,"I started use Truvada, when Intelence affected...",2011-09-30,10.0,drugs.com,NRT
8,Truvada,HIV,"My viral load has gone for 180,000 to undetect...",2011-05-19,10.0,drugs.com,NRT
9,Truvada,HIV,After one year on Truvada in combination with ...,2010-04-19,9.0,drugs.com,NRT


In [114]:
# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [142]:
# Variables for holding sentiments
compound_list = []
positive_list = []
neutral_list = []
negative_list = []        
score = []

vader_review_analysis = drugs_scrape_df
# Loop through all tweets
for review in vader_review_analysis['Reviews']:

    # Run Vader Analysis on each tweet
    results = analyzer.polarity_scores(review)
    compound = results["compound"]
    pos = results["pos"]
    neu = results["neu"]
    neg = results["neg"]

    # Add each value to the appropriate list
    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)
    
    #Make Vader "Score" that is tangible for the public
    if compound <= (-.6):
        score.append('Very Negative')
    elif compound <= (-.2):
        score.append('Negative')
    elif compound <= .2:
        score.append('Neutral')
    elif compound <= .6:
        score.append('Positive')
    else:
        score.append('Very Positive')
    
#Add to dataframe
vader_review_analysis['Compound'] = compound_list
vader_review_analysis['Positive'] = positive_list
vader_review_analysis['Neutral'] = neutral_list
vader_review_analysis['Negative'] = negative_list
vader_review_analysis['Score'] = score
    
# # Print the Averages
# print(f"User: {target_user}")
# print(f"Compound: {np.mean(compound_list):.3f}")
# print(f"Positive: {np.mean(positive_list):.3f}")
# print(f"Neutral: {np.mean(neutral_list):.3f}")
# print(f"Negative: {np.mean(negative_list):.3f}")

# print(compound_list,positive_list,negative_list,neutral_list)
print(vader_review_analysis)

         Drug Reason                                            Reviews  \
0    Truvada     HIV  Doctor put me  truvada as part of my medicatio...   
1    Truvada     HIV  No side effects experienced in almost two year...   
2    Truvada     HIV  I`ve been on truvada daily for more than 5 yea...   
3    Truvada     HIV  I have been taking  Truvuda and Edurant combin...   
4    Truvada     HIV  Been on Truvada/Norvir/Prezista combination fo...   
5    Truvada     HIV  I started Truvada once a day along with Isentr...   
6    Truvada     HIV  I have been on the combination of Truvada/Prez...   
7    Truvada     HIV  I started use Truvada, when Intelence affected...   
8    Truvada     HIV  My viral load has gone for 180,000 to undetect...   
9    Truvada     HIV  After one year on Truvada in combination with ...   
10   Truvada     HIV  I've been using Truvada for about three months...   
11   Atripla     HIV  I was on atripla for 10 years and my doctor be...   
12   Atripla     HIV  vir

In [135]:
#Make stats_by_drug dataframe
stats_by_drug = vader_review_analysis.groupby('Drug').mean()
stats_by_drug['Counts'] = vader_review_analysis.groupby('Drug').size()
stats_by_drug

,Ratings,Compound,Positive,Neutral,Negative,Counts
Drug,,,,,,
Descovy,10.000000,0.0772,0.094,0.824,0.082,2
Norvir,1.000000,0.0772,0.094,0.824,0.082,1
Atripla,9.086957,0.0772,0.094,0.824,0.082,125
Truvada,7.241379,0.0772,0.094,0.824,0.082,33


In [ ]:
#Make stats_by_classify dataframe
stats_by_classify = vader_review_analysis.groupby('Classify').mean()
stats_by_classify['Counts'] = vader_review_analysis.groupby('Drug').size()
stats_by_classify

In [ ]:
#Make stats_reason_compare (Truvada and Descovy ONLY!)
class_averages_df = vader_review_analysis.groupby('Classify').mean()
vader_averages_df['Counts'] = vader_review_analysis.groupby('Drug').size()
class_averages_df